# Import Packages

To begin with we need to import packages to work with, these being:

1. harmonic - for bayesian evidence computation.
2. numpy - for basic math functions.
3. emcee - for MCMC sampling (can be replaced by any preferred sampling package). 


In [2]:
import numpy as np
import emcee
import matplotlib.pyplot as plt
from matplotlib import cm
from functools import partial
import sys
sys.path.insert(0, '/Users/matt/Downloads/Software/harmonic')
import harmonic as hm
sys.path.append("../examples")
import utils

[2020-07-12 17:50:37,122] [Harmonic] [CRITICAL]: Using config from /Users/matt/Downloads/Software/harmonic/logs/logging.yaml
[2020-07-12 17:50:37,301] [Harmonic] [CRITICAL]: Using config from /Users/matt/Downloads/Software/harmonic/logs/logging.yaml


# Define Bayesian Posterior Function

Now we will need to define the log-posterior function we are interested in. Here we consider perhaps the most basic case of a n-dimensional uniform prior with a simple Gaussian likelihood defined by 

$$
\mathcal{L}(\boldsymbol{x}) = \frac{1}{2\pi \: \text{det} \boldsymbol{\Sigma}^{-\frac{1}{2}}}e^{-\frac{1}{2} \boldsymbol{x}^T\boldsymbol{\Sigma}^{-1}\boldsymbol{x}},
$$
for $\boldsymbol{x} \in \mathbb{R}^n$. This is simply coded as


In [3]:
def ln_posterior(x, inv_cov):
    """
    Compute log_e of posterior.
    Args: 
        - x: 
            Position at which to evaluate posterior.
        - inv_cov: 
            Inverse covariance matrix.      
    Returns:
        - double: 
            Value of Gaussian at specified point.
    """
    
    return -np.dot(x,np.dot(inv_cov,x))/2.0

for non-diagonal covariance matrix $\boldsymbol{\Sigma}$ with randomized entries computed by the following function

In [4]:
def init_cov(ndim):
    """
    Initialise random non-diagonal covariance matrix.
    Args: 
        - ndim: 
            Dimension of Gaussian.           
    Returns:
        - cov: 
            Covariance matrix of shape (ndim,ndim).
    """
    
    cov = np.zeros((ndim,ndim))
    diag_cov = np.ones(ndim) + np.random.randn(ndim)*0.1
    np.fill_diagonal(cov, diag_cov)
    
    for i in range(ndim-1):
        cov[i,i+1] = (-1)**i * 0.5*np.sqrt(cov[i,i]*cov[i+1,i+1])
        cov[i+1,i] = cov[i,i+1]
    return cov 


# Analytic Evidence

Finally we require the analytic evidence to compare our estimate against. Thankfully this is straightforward in this setting and is given by 

$$
\int_{-\infty}^{\infty}\int_{-\infty}^{\infty}\mathcal{L}(\boldsymbol{x})d\boldsymbol{x} = \frac{(2\pi)^{\frac{n}{2}}}{\sqrt{\:\text{det}\boldsymbol{\Sigma}}}
$$

which can be coded in python as 

In [5]:
def ln_analytic_evidence(ndim, cov):
    """
    Compute analytic evidence for nD Gaussian.
    Args:
        - ndim: 
            Dimension of Gaussian.
        - cov: 
            Covariance matrix.
    Returns:
        - double:
            Analytic evidence.
    """
    
    ln_norm_lik = 0.5*ndim*np.log(2*np.pi) + 0.5*np.log(np.linalg.det(cov))
    return ln_norm_lik

# Compute samples using Emcee

In typical fashion we now need to collect some samples of the posterior using whichever MCMC package you wish to use. Here we'll collect samples using the [emcee](https://emcee.readthedocs.io/en/stable/) package.

First we will need to define and initialize some variables:

In [12]:
# Define parameters for emcee sampling
ndim = 2                                     # number of dimensions
nchains = 100                                # total number of chains to compute
samples_per_chain = 5000                     # number of samples per chain
nburn = 500                                  # number of samples to discard as burn in

# Initialize random seed
np.random.seed(10)

# Create random non-diagonal inverse covariance
cov = init_cov(ndim)
inv_cov = np.linalg.inv( cov )   

Now we need to run the sampler to collect samples:

In [7]:
# Set initial random position and state
pos = np.random.rand(ndim * nchains).reshape((nchains, ndim))
rstate = np.random.get_state()

# Instantiate and execute Sampler 
sampler = emcee.EnsembleSampler(nchains, ndim, ln_posterior, args=[inv_cov])
(pos, prob, state) = sampler.run_mcmc(pos, samples_per_chain, rstate0=rstate) 

# Collect samples into contiguous numpy arrays (discarding burn in)
samples = np.ascontiguousarray(sampler.chain[:,nburn:,:])
lnprob = np.ascontiguousarray(sampler.lnprobability[:,nburn:])

# Harmonic Code

Here is where one would normally start (*i.e.* already with chains ready to compute the evidence).

## Collating samples using harmonic.chains class

Now we simply need to configure the chains into a harmonic friendly shape which we do by:

In [8]:
# Instantiate harmonic's chains class 
chains = hm.Chains(ndim)
chains.add_chains_3d(samples, lnprob)

# Split the chains into the ones which will be used to train/test the machine learning model
chains_train, chains_test = hm.utils.split_data(chains, training_proportion=0.05)

## Train the machine learning model

Now take the chains_train from the above code and use them to train the model (here we have selected the HyperSphere model for simplicity):

In [9]:
# Define the model domain over which to optimize the hyper-parameters
domains = [np.sqrt(ndim-1)*np.array([1E-1,1E0])]

# Now simply instantiate the model class you wish and train it
model = hm.model.HyperSphere(ndim, domains)
fit_success, objective = model.fit(chains_train.samples, chains_train.ln_posterior) 

## Compute the Bayesian evidence

Finally we simply compute the Learnt Harmonic Mean estimator by:

In [10]:
# Instantiate harmonic's evidence class
ev = hm.Evidence(chains_test.nchains, model)

# Pass the evidence class the test chains and compute the evidence!
ev.add_chains(chains_test)
ln_evidence, ln_evidence_std = ev.compute_ln_evidence()

## Error analysis

Lets take a look at what the error on our estimate is!

In [19]:
# Compute the analytic evidence
ln_evidence_analytic = ln_analytic_evidence(ndim, cov)

print('Analytic value: {} || Learnt Harmonic Mean value: {}'.format(ln_evidence_analytic, ln_evidence))

# Compare to the learnt harmonic mean estimate 
abs_error = np.abs(ln_evidence_analytic - ln_evidence)
percent_error = 100.0 * abs_error / ln_evidence_analytic

# Compare the variance provided by the learnt harmonic mean estimate
sigma_error = np.log(np.exp(ln_evidence) + np.exp(ln_evidence_std)) - ln_evidence

sigma_count = abs_error / sigma_error

print('Number of sigma away from true value: {}'.format(sigma_count))


# N.B: sigma_count represents the number of standard deviations between the estimated 
#     and analytic results.

Analytic value: 1.791083315460074 || Learnt Harmonic Mean value: 1.7932610258353037
Number of sigma away from true value: 0.5226588159689041
